In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-mxjkzm9q
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-mxjkzm9q
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=308296755d14180c0bc9969c7f275d0f78b3c962c5daa51761a27f8beb97c6a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-cycfes0h/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
!pwd

/content


In [ ]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %shell  %store  %sx  %system  %tb  %tensorflow_version  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%bigquery  %%capture  %%cu  %%cuda  %%cuda_run  %%debug  %%file  %%html  %%javascri

In [24]:
%%writefile sequential.cpp
#include<stdio.h>
#include <bits/stdc++.h>

using namespace std;

void get_adj_matrix(float* graph, int n, float d, FILE *inputFilePtr ){

    if ( inputFilePtr == NULL )  {
        printf( "input.txt file failed to open." );
        return ;
    }

    int m, indexing;
    
    fscanf(inputFilePtr, "%d", &m);
    fscanf(inputFilePtr, "%d", &indexing);

    
    for(int i = 0; i< n ; i++){
    
        for(int j = 0; j< n; ++j){
            graph[i* n + j] = (1 - d)/float(n);
        }
    }

    while(m--){
        int source, destin;
        fscanf(inputFilePtr, "%d", &source);
        fscanf(inputFilePtr, "%d", &destin);
        if (indexing == 0){
            graph[destin* n + source] += d* 1.0  ;
        }
        else{
            graph[(destin - 1)* n + source - 1] += d* 1.0;
        }
    }
}
void manage_adj_matrix(float* graph, int n){

    for(int j = 0; j < n; ++j){
        float sum = 0.0;

        for (int i = 0; i< n; ++i){
            sum += graph[i* n + j];
        }

        for (int i = 0; i < n; ++i){
            if (sum != 0.0){
                graph[i*n + j] /= sum;
            }
            else{
                graph[i*n + j] = (1/(float)n);
            }
        }
    }

}
float norm(float *vect, int n){
    float ans = 0.0;
    for (int i = 0; i < n; ++i){
        ans += abs(vect[i]);
    }
    return ans;
}

void power_method(float *graph, float *r, int n, int max_iter = 1000, float eps = 0.000001 ){
   
    float* r_last = (float*) malloc(n * sizeof(float));
    

    for(int i = 0; i< n; ++i){
        r[i] = (1/(float)n);
    }

    while(max_iter--){
        for(int i = 0; i< n; ++i){
            r_last[i] = r[i];
        }
        for(int i = 0; i< n; ++i){
            float sum = 0.0;

            for (int j = 0; j< n; ++j){
                sum += r_last[j] * graph[i* n + j];
            }

            r[i] = sum;

        }

        for(int i = 0; i< n; ++i){
            r_last[i] -= r[i];
        }

        if(norm(r_last, n) < eps){
            return;
        }

    }
    return;
}

void top_nodes(float *r, int n, int count = 3){

    priority_queue<pair<float, int>> pq;

    for(int i = 0; i< n; ++i){
        pq.push(make_pair(r[i], i+ 1));
    }
    int rank =1;
    while(rank <= count){
        printf("Rank %d Node is %d\n", rank, pq.top().second);
        rank++;
        pq.pop();
    }

}

int main(int argc, char** argv){

    clock_t start, end;

    FILE *inputFilePtr;

    char * inputfile = argv[1];
    inputFilePtr = fopen(inputfile, "r");

    int n; 
    fscanf(inputFilePtr, "%d", &n);

    float d = 0.85; 

    float* graph = (float*)malloc(n *n* sizeof(float));


    float* r = (float*) malloc(n * sizeof(float));

   
    get_adj_matrix(graph, n, d, inputFilePtr);

    start = clock();

    manage_adj_matrix(graph, n);
    power_method(graph, r, n);
    //top_nodes(r, n);

    end = clock();

    printf("Time taken :%f for sequential implementation with %d nodes.\n", float(end - start), n);
    return 0;
}

Overwriting sequential.cpp


In [25]:
%%shell
cd /content/
g++ sequential.cpp
./a.out 100_2.txt
./a.out 100.txt
./a.out 200.txt
./a.out 400.txt
./a.out 800.txt
./a.out 1000.txt
./a.out 5000.txt
./a.out 10000.txt
./a.out 16000.txt

Time taken :392.000000 for sequential implementation with 100 nodes.
Time taken :239.000000 for sequential implementation with 100 nodes.
Time taken :6485.000000 for sequential implementation with 200 nodes.
Time taken :3366.000000 for sequential implementation with 400 nodes.
Time taken :20242.000000 for sequential implementation with 800 nodes.
Time taken :2970832.000000 for sequential implementation with 1000 nodes.
Time taken :67973952.000000 for sequential implementation with 4772 nodes.
Time taken :67280016.000000 for sequential implementation with 9664 nodes.
Time taken :26965332.000000 for sequential implementation with 16062 nodes.


In [10]:
%%shell
cd /content/
g++ sequential.cpp

./a.out 18000.txt


tcmalloc: large alloc 1364418560 bytes == 0x55a37d16c000 @  0x7f8f2863c1e7 0x55a37b2c3377 0x7f8f27c98bf7 0x55a37b2c2b6a
Rank 1 Node is 4949
Rank 2 Node is 11833
Rank 3 Node is 5461
Time taken :1030231744.000000 for sequential implementation with 18469 nodes.


In [16]:
%%writefile parallel.cu
#include<stdio.h>
#include <bits/stdc++.h>
#include<cuda.h>
#include <thrust/reduce.h>
#include <thrust/sort.h>
#include <thrust/execution_policy.h>

using namespace std;


void get_adj_matrix(float* graph, int n, float d, FILE *inputFilePtr ){

    if ( inputFilePtr == NULL )  {
        printf( "input.txt file failed to open." );
        return ;
    }

    int m, indexing;
    
    fscanf(inputFilePtr, "%d", &m);
    fscanf(inputFilePtr, "%d", &indexing);

    
    for(int i = 0; i< n ; i++){
    
        for(int j = 0; j< n; ++j){
            graph[i* n + j] = (1 - d)/float(n);
        }
    }

    while(m--){
        int source, destin;
        fscanf(inputFilePtr, "%d", &source);
        fscanf(inputFilePtr, "%d", &destin);
        if (indexing == 0){
            graph[destin* n + source] += d* 1.0  ;
        }
        else{
            graph[(destin - 1)* n + source - 1] += d* 1.0;
        }
    }
}

__global__ void manage_adj_matrix(float* gpu_graph, int n){
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if(id < n){
        float sum = 0.0;

        for (int i = 0; i< n; ++i){
            sum += gpu_graph[i* n + id];
        }

        for (int i = 0; i < n; ++i){
            if (sum != 0.0){
                gpu_graph[i* n + id] /= sum;
            }
            else{
                gpu_graph[i* n + id] = (1/(float)n);
            }
        }
    }
}

__global__ void initialize_rank(float* gpu_r, int n){
    int id = blockIdx.x * blockDim.x + threadIdx.x;

    if(id < n){
        gpu_r[id] = (1/(float)n);
    }
}

__global__ void store_rank(float* gpu_r,float* gpu_r_last, int n){
    int id = blockIdx.x * blockDim.x + threadIdx.x;

    if(id < n){
        gpu_r_last[id] = gpu_r[id];
    }
}

__global__ void matmul(float* gpu_graph, float* gpu_r, float* gpu_r_last, int n){
    int id = blockIdx.x * blockDim.x + threadIdx.x;

    if(id < n){
        float sum = 0.0;

        for (int j = 0; j< n; ++j){
            sum += gpu_r_last[j] * gpu_graph[id* n + j];
        }

        gpu_r[id] = sum;
    }
}

__global__ void rank_diff(float* gpu_r,float* gpu_r_last, int n){
    int id = blockIdx.x * blockDim.x + threadIdx.x;

    if(id < n){
        gpu_r_last[id] = abs(gpu_r_last[id] - gpu_r[id]);
    }
}

__global__ void init_pair_array(pair<float, int>* gpu_r_nodes, float * gpu_r, int n){
    int id = blockIdx.x * blockDim.x + threadIdx.x;

    if(id < n){
        gpu_r_nodes[id].first = gpu_r[id];
        gpu_r_nodes[id].second = id + 1;
    }
}


void power_method(float *graph, float *r, int n, int nblocks, int BLOCKSIZE, int max_iter = 1000, float eps = 0.000001 ){
   
    float* r_last = (float*) malloc(n * sizeof(float));
    
    float* gpu_graph;
    cudaMalloc(&gpu_graph, sizeof(float)*n*n);
    cudaMemcpy(gpu_graph, graph, sizeof(float)*n*n, cudaMemcpyHostToDevice);

    float* gpu_r;
    cudaMalloc(&gpu_r, sizeof(float)*n);
    //cudaMemcpy(gpu_r, r, sizeof(float)*n, cudaMemcpyHostToDevice);

    float* gpu_r_last;
    cudaMalloc(&gpu_r_last, sizeof(float)*n);
    //cudaMemcpy(gpu_r_last, r_last, sizeof(float)*n, cudaMemcpyHostToDevice);



    initialize_rank<<<nblocks, BLOCKSIZE>>>(gpu_r, n);
    cudaDeviceSynchronize();



    while(max_iter--){

        store_rank<<<nblocks, BLOCKSIZE>>>(gpu_r, gpu_r_last, n);
        cudaDeviceSynchronize();

        matmul<<<nblocks, BLOCKSIZE>>>(gpu_graph, gpu_r, gpu_r_last, n);
        cudaDeviceSynchronize();
        
        rank_diff<<<nblocks, BLOCKSIZE>>>(gpu_r, gpu_r_last, n);
        cudaDeviceSynchronize();

        cudaMemcpy(r_last, gpu_r_last, n* sizeof(float), cudaMemcpyDeviceToHost);
        float result = thrust::reduce( r_last, r_last + n);

        if(result < eps){
            cudaMemcpy(r, gpu_r, n* sizeof(float), cudaMemcpyDeviceToHost);
            return;
        }
    }
    cudaMemcpy(r, gpu_r, n* sizeof(float), cudaMemcpyDeviceToHost);
    return;
}

void top_nodes(float* r, int n, int nblocks, int BLOCKSIZE, int count = 3){

    pair<float, int> *r_nodes = (pair<float, int> *) malloc ( n * sizeof (pair<float, int>) );
    pair<float, int> *gpu_r_nodes;

    cudaMalloc(&gpu_r_nodes, n * sizeof (pair<float, int>));

    float* gpu_r;
    cudaMalloc(&gpu_r, sizeof(float)*n);
    cudaMemcpy(gpu_r, r, sizeof(float)*n, cudaMemcpyHostToDevice);

    init_pair_array<<<nblocks, BLOCKSIZE>>>(gpu_r_nodes, gpu_r, n);

    cudaMemcpy(r_nodes, gpu_r_nodes, n * sizeof (pair<float, int>), cudaMemcpyDeviceToHost);

    thrust::sort(thrust::host, r_nodes, r_nodes + n);

    int rank =1;
    while(rank <= count){
        printf("Rank %d Node is %d\n", rank, r_nodes[n - rank].second);
        rank++;
    }
}

int main(int argc, char** argv){

    clock_t start, end;

    FILE *inputFilePtr;

    char * inputfile = argv[1];

    char * bsize = argv[2];
    int BLOCKSIZE = atoi(bsize);

    inputFilePtr = fopen(inputfile, "r");

    int n; 

    fscanf(inputFilePtr, "%d", &n);
    int nblocks = ceil(float(n) / BLOCKSIZE);

    float* graph = (float*)malloc(n*n*sizeof(float));
    float* r = (float*) malloc(n * sizeof(float));

    float d = 0.85;

    get_adj_matrix(graph, n, d, inputFilePtr);

    float* gpu_graph;
    cudaMalloc(&gpu_graph, sizeof(float)*n*n);
    cudaMemcpy(gpu_graph, graph, sizeof(float)*n*n, cudaMemcpyHostToDevice);
    
    start = clock();

    manage_adj_matrix<<<nblocks, BLOCKSIZE>>>(gpu_graph, n);
    cudaMemcpy(graph, gpu_graph, sizeof(float)*n*n, cudaMemcpyDeviceToHost);

    power_method(graph, r, n, nblocks, BLOCKSIZE );

    //top_nodes(r, n, nblocks, BLOCKSIZE);

    end = clock();

    printf("Time taken :%f for parallel implementation with %d nodes and %d threads per block.\n", float(end - start), n, BLOCKSIZE);
    return 0;
}

Overwriting parallel.cu


In [17]:
%%shell
cd /content/
nvcc parallel.cu
./a.out 100_2.txt 32
./a.out 100.txt 32
./a.out 200.txt 32
./a.out 400.txt 32
./a.out 800.txt 32
./a.out 1000.txt 32
./a.out 5000.txt 32
./a.out 10000.txt 32
./a.out 16000.txt 32
./a.out 18000.txt 32

/usr/local/cuda/bin/../targets/x86_64-linux/include/thrust/system/detail/sequential/assign_value.h(36): warning: calling a __host__ function from a __host__ __device__ function is not allowed
          detected during:
            instantiation of "void thrust::system::detail::sequential::assign_value(thrust::system::detail::sequential::execution_policy<DerivedPolicy> &, Pointer1, Pointer2) [with DerivedPolicy=thrust::system::cpp::detail::tag, Pointer1=thrust::pointer<std::pair<float, int>, thrust::system::cpp::detail::par_t, thrust::use_default, thrust::use_default>, Pointer2=const std::pair<float, int> *]" 
/usr/local/cuda/bin/../targets/x86_64-linux/include/thrust/detail/reference.inl(183): here
            instantiation of "void thrust::reference<Element, Pointer, Derived>::strip_const_assign_value(const System &, OtherPointer) [with Element=std::pair<float, int>, Pointer=thrust::pointer<std::pair<float, int>, thrust::system::cpp::detail::par_t, thrust::use_default, thrust::use_def

In [ ]:
%%shell
cd /content/
nvcc parallel.cu

./a.out 18000.txt 32

In [18]:
%%shell
cd /content/
nvcc parallel.cu

./a.out 10000.txt 8
./a.out 10000.txt 16
./a.out 10000.txt 32
./a.out 10000.txt 64
./a.out 10000.txt 128
./a.out 10000.txt 256
./a.out 10000.txt 512
./a.out 10000.txt 1024

/usr/local/cuda/bin/../targets/x86_64-linux/include/thrust/system/detail/sequential/assign_value.h(36): warning: calling a __host__ function from a __host__ __device__ function is not allowed
          detected during:
            instantiation of "void thrust::system::detail::sequential::assign_value(thrust::system::detail::sequential::execution_policy<DerivedPolicy> &, Pointer1, Pointer2) [with DerivedPolicy=thrust::system::cpp::detail::tag, Pointer1=thrust::pointer<std::pair<float, int>, thrust::system::cpp::detail::par_t, thrust::use_default, thrust::use_default>, Pointer2=const std::pair<float, int> *]" 
/usr/local/cuda/bin/../targets/x86_64-linux/include/thrust/detail/reference.inl(183): here
            instantiation of "void thrust::reference<Element, Pointer, Derived>::strip_const_assign_value(const System &, OtherPointer) [with Element=std::pair<float, int>, Pointer=thrust::pointer<std::pair<float, int>, thrust::system::cpp::detail::par_t, thrust::use_default, thrust::use_def